In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.spatial.distance as dist
import matplotlib as mpl
output_dir="output"
import os

from perdiver.distances import *
import perdiver.perdiver as perdiver

In [ ]:
from navground import sim, core

number = 30
yaml = f"""
steps: 10000
time_step: 0.1
save_directory: ''
record_pose: true
record_twist: true
scenario:
  type: Cross
  agent_margin: 0.2
  side: 28
  target_margin: 0.6
  tolerance: 0.4
  groups:  
    -
      type: thymio
      number: {number}
      control_period: 0.1
      behavior:
        type: HL
        safety_margin: 0.25 
        horizon: 3
        barrier_angle: 1
      radius: 0.25
      kinematics:
        type: 2WDiff
        max_speed: 1.0
        wheel_axis: 2
      state_estimation:
        type: Bounded
        range: 2.0 
"""
experiment = sim.load_experiment(yaml)
experiment.run()

In [ ]:
run = experiment.runs[0]
ps = run.poses[:,:,[0,1]]
twists = run.twists[:,:,:2] # ignore angular speeds

In [ ]:
weight = 4

In [ ]:
# def trajectory_distance_weighted_velocities(positions, velocities, weight):
#     assert(len(positions)>0)
#     assert(len(positions)==len(velocities))
#     positions_velocities_list = []
#     for idx, points in enumerate(positions):
#         positions_velocities_list.append(np.hstack((points, velocities[idx]*weight)))
#     distances_list = []
#     for idx, points_vel in enumerate(positions_velocities_list):
#         # Compare trajectories at same time
#         points_vel_compare = positions_velocities_list[idx]
#         distances_list.append(dist.cdist(points_vel, points_vel_compare, "minkowski", p=2))
#     # end for
#     distances_arr = np.array(distances_list)
#     return np.min(distances_arr, axis=0)

In [ ]:
dist_result = perdiver.trajectory_distance_weighted_velocities(ps[:5], twists[:5], weight)

In [ ]:
# def distances_weighted_velocities(points, velocities, weight):
#     points_velocities = np.hstack((points, velocities*weight))
#     return dist.squareform(dist.pdist(points_velocities, "minkowski", p=2))

In [ ]:
# def plot_matching_diagram_trajectories(positions, velocities, weight, ax, shift_time, color="blue"):
#     # Compute X, Z barcodes and matching
#     half_shift=int(shift_time/2)
#     Dist_X = trajectory_distance_weighted_velocities(positions[:half_shift], velocities[:half_shift], weight)
#     Dist_Y = trajectory_distance_weighted_velocities(positions, velocities, weight)
#     Dist_Z = np.minimum(Dist_X, Dist_Y)
#     idx_S = list(range(Dist_X.shape[0]))
#     # Compute matching from X to Z
#     ibfm_out = ibfm.get_IBloFunMatch_subset(Dist_X, Dist_Z, idx_S, output_dir, max_rad=-1, num_it=1, store_0_pm=True, points=False, max_dim=1)
#     # Plot 0 persistence diagram of matching 
#     match_diagram = []
#     for idx, bar_X in enumerate(ibfm_out["S_barcode_0"]):
#         idx_match = ibfm_out["induced_matching_0"][idx]
#         bar_Z = ibfm_out["X_barcode_0"][idx_match]
#         match_diagram.append([bar_X[1], bar_Z[1]])
#     # end for
#     match_diagram = np.array(match_diagram)
#     # Plot matching diagram
#     ax.scatter(match_diagram[:,0], match_diagram[:,1], color=color)
#     ax.plot([0,np.max(match_diagram)*1.1], [0,np.max(match_diagram)*1.1], color="gray")

In [ ]:
start_step_list = list(range(0,2400, 500))
fig, ax = plt.subplots(ncols=len(start_step_list), figsize=(6*(len(start_step_list)),6))
xmax, ymax = 0,0
for idx_start, start_step in enumerate(start_step_list):
    shift_time=50
    X, Y = ps[idx_start], ps[idx_start+shift_time]
    vel_X, vel_Y = twists[idx_start], twists[idx_start+shift_time]
    Dist_X = perdiver.distances_weighted_velocities(X, vel_X, weight)
    Dist_Y = perdiver.distances_weighted_velocities(Y, vel_Y, weight)
    Dist_Z = np.minimum(Dist_X, Dist_Y)
    perdiver.plot_matching_diagram(Dist_X, Dist_Y, Dist_Z, ax[idx_start])
    # plot_matching_diagram_trajectories(ps[start_step:start_step+shift_time], twists[start_step:start_step+shift_time], weight, ax[idx_start], shift_time, color=mpl.colormaps["GnBu"](idx/len(steplist)))
    ax[idx_start].set_title(f"{start_step}")
    xmax = max(xmax, ax[idx_start].get_xlim()[1])
    ymax = max(ymax, ax[idx_start].get_ylim()[1])

for idx_start, start_step in enumerate(start_step_list):
    ax[idx_start].plot([0,max(xmax, ymax)], [0, max(xmax, ymax)], color="gray")
    ax[idx_start].set_xlim(0,xmax)
    ax[idx_start].set_ylim(0,ymax)

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(7,4))
step = 200
shift_time = 100
X, Y = ps[step], ps[step+shift_time]
vel_X, vel_Y = twists[step], twists[step+shift_time]
Dist_X = perdiver.distances_weighted_velocities(X, vel_X, weight)
Dist_Y = perdiver.distances_weighted_velocities(Y, vel_Y, weight)
Dist_Z = np.minimum(Dist_X, Dist_Y)
ibfm_out = perdiver.plot_matched_barcodes(Dist_X, Dist_Z, ax, fig)
plt.tight_layout()
plt.savefig(f"matching_diagram_cross_{number}.png")

In [ ]:
fig, ax = plt.subplots(figsize=(5,5))
step = 200
shift_time = 100
X, Y = ps[step], ps[step+shift_time]
vel_X, vel_Y = twists[step], twists[step+shift_time]
Dist_X = perdiver.distances_weighted_velocities(X, vel_X, weight)
Dist_Y = perdiver.distances_weighted_velocities(Y, vel_Y, weight)
Dist_Z = np.minimum(Dist_X, Dist_Y)
perdiver.plot_matching_diagram(Dist_X, Dist_Y, Dist_Z, ax)
plt.tight_layout()
plt.savefig(f"per_div_cross_{number}.png")

In [ ]:
start_step_list = list(range(0,900, 10))
shift_time=50
fig, ax = plt.subplots(figsize=(8,6))
for idx, step in enumerate(start_step_list):
    X, Y = ps[step], ps[step+shift_time]
    vel_X, vel_Y = twists[step], twists[step+shift_time]
    Dist_X = perdiver.distances_weighted_velocities(X, vel_X, weight)
    Dist_Y = perdiver.distances_weighted_velocities(Y, vel_Y, weight)
    Dist_Z = np.minimum(Dist_X, Dist_Y)
    perdiver.plot_matching_diagram(Dist_X, Dist_Y, Dist_Z, ax, color=mpl.colormaps["GnBu"](idx/len(start_step_list)), max_val_diag=13)
    
# Colorbar 
norm = mpl.colors.Normalize(start_step_list[0], vmax=start_step_list[-1])
cmap = mpl.colormaps["GnBu"]
mappable = mpl.cm.ScalarMappable(norm=norm, cmap=cmap)
plt.colorbar(mappable=mappable, ax=ax)
plt.tight_layout()
plt.savefig(f"perdiver_diagram_cross_{number}.png")

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))
X = ps[900]
ax.scatter(X[:,0], X[:,1], c="black", marker="o", s=20)